# MSEE UQ short course:  The $\texttt{UQpy}$ library

Application of surrogate modeling using the $\texttt{UQpy}$ module $\texttt{Surrogates}$.

Detailed instructions on how to use this module can be found in the $\texttt{UQpy}$ documentation.

https://uqpyproject.readthedocs.io/en/latest/surrogates_doc.html

# Exercise 1

### Polynomial Chaos

Build a PCE surrogate for 1-D Bouc-Wen $\texttt{Python}$ model. In this case, randomness is assumed in the systems' parameter $r_{0}$ which is assumed to be uniformly distributed in the range [0.5, 3.5]. The response of interest is the maximum displacement $z(t)$ of the system.

### Step 1

Create a distribution object for the random variable (see Day 1 activities).

In [ ]:
# Solution
from UQpy.Distributions import Uniform

distribution=Uniform(0.5, 3.0)

### Step 2

Create a $\texttt{MCS}$ object (see Day 1 activities):
- generate 30 realizations of $r_0$ that will serve as our training input set.
- generate 20 realizations of $r_0$ that will serve as our validation input set.

In [ ]:
# Solution
from UQpy.SampleMethods import MCS

training_sampling = MCS(dist_object=distribution, nsamples=30)
training_samples=training_sampling.samples

validation_sampling = MCS(dist_object=distribution, nsamples=20)
validation_samples = validation_sampling.samples

### Step 3

Run the 1-D Bouc-Wen computational model using $\texttt{RunModel}$ module of $\texttt{UQpy}$ (see Day 1 activities):
- for the training input set.
- for the validation input set.

In [ ]:
# Solution
from UQpy.RunModel import RunModel
from UQpy.Distributions import Uniform
from UQpy.SampleMethods import MCS
import numpy as np
import matplotlib.pyplot as plt
from UQpy.Surrogates import *

boucwen = RunModel(model_script='model_1D.py', model_object_name='boucwen_runmodel', var_names=['r0'])
boucwen.run(samples=training_samples)

qoi= boucwen.qoi_list

maximum_displacement = boucwen.qoi_list[:30]

boucwen.run(samples=validation_samples)

maximum_displacement_validation=boucwen.qoi_list[-20:]

### Step 4

Create the PCE surrogate to approximate the response function (i.e., maximum displacement $\max(z(t))$ of the system. Compute the PCE coefficients using least square regression. Compute the validation error.

### Step 4.1:

Import libraries  $\texttt{PCE}$, $\texttt{Polynomials}$ and $\texttt{PolyChaosLstsq}$, from $\texttt{UQpy.Surrogates}$ module.


In [ ]:
# Solution
from UQpy.Surrogates import PCE, PolyChaosLstsq, Polynomials

### Step 4.2:

Define a 'polynomials' object using the $\texttt{Polynomials}$ class. Provide as input to the class the distribution object of the random variable, and the maximum  PCE order p=1.

In [ ]:
# Solution
polys = Polynomials(dist_object=distribution, degree=1)

### Step 4.3:

Define a least squares regression object using the $\texttt{PolyChaosLstsq}$ class. Provide as input to the class the polynomials object you created in the previous step. 

In [ ]:
# Solution
lstsq = PolyChaosLstsq(poly_object=polys)

### Step 4.4:

Define a pce object using the $\texttt{PCE}$ class. Provide as input method the regression object created in the previous step.

In [ ]:
# Solution
pce = PCE(method=lstsq)

### Step 4.5:

Calculate the coefficients using the $\texttt{fit}$ method of the $\texttt{PCE}$ object. Provide the training data set (input-output pairs).

In [ ]:
# Solution
pce.fit(training_samples,np.array(maximum_displacement).reshape(30,1))

### Step 4.6:

Predic the response for the validation input set using the $\texttt{predict}$ method of the $\texttt{PCE}$ object.

In [ ]:
# Solution
prediction_sampling=MCS(dist_object=[distribution], nsamples=100,  verbose=True)
prediction_results=pce.predict(prediction_sampling.samples)

### Step 4.7:

Compute the validation error:
- Import the $\texttt{ErrorEstimation}$ class from $\texttt{UQpy.Surrogates}$ module.
- Compute the validation error using the $\texttt{validation}$ method.

In [ ]:
# Solution
from UQpy.Surrogates import ErrorEstimation
error = ErrorEstimation(surr_object=pce)
print('Error from least squares regression is: ', error.validation(validation_sampling.samples, np.array(maximum_displacement_validation)))

### Activities

1. Build the PCE surrogate for the 1-D Bouc-Wen model for:
    - For 100 training data.
    - maximum PCE order p=4.
    - Ridge regression methods.
    For this case, estimate the first two moments (mean and variance) of the PCE surrogate.
    
    
2. Build a PCE surrogate for the 2-D contact sphere model. In this case, randomness is assumed in the systems' parameter $\texttt{k}\sim\mathcal{N}(10^5, 2\times 10^4)$ and $\texttt{f0}\sim \mathcal{U}(0.01, 0.1)$. The response of interest is the maximum absolute value of the displacement field at the identation point.

In [ ]:
# Activity 1 - Solution
from UQpy.RunModel import RunModel
from UQpy.Distributions import Uniform
from UQpy.SampleMethods import MCS
import numpy as np
import matplotlib.pyplot as plt
from UQpy.Surrogates import *
from UQpy.SampleMethods import MCS
from UQpy.Distributions import Uniform

distribution=Uniform(0.5, 3.0)


training_sampling = MCS(dist_object=distribution, nsamples=100)
training_samples=training_sampling.samples

boucwen = RunModel(model_script='model_1D.py', model_object_name='boucwen_runmodel', var_names=['r0'])
boucwen.run(samples=training_samples)

maximum_displacement = boucwen.qoi_list[:100]

from UQpy.Surrogates import PCE, PolyChaosRidge, Polynomials

polys = Polynomials(dist_object=distribution, degree=4)
lstsq = PolyChaosRidge(poly_object=polys)
pce = PCE(method=lstsq)

pce.fit(training_samples,np.array(maximum_displacement).reshape(100,1))

prediction_sampling=MCS(dist_object=[distribution], nsamples=100,  verbose=True)
prediction_results=pce.predict(prediction_sampling.samples)

from UQpy.Surrogates import MomentEstimation
print('Moments from Ridge regression :', MomentEstimation(surr_object=pce).get())

In [ ]:
# Activity 2 - Solution
from UQpy.RunModel import RunModel
from UQpy.Distributions import Uniform, Normal, JointInd
from UQpy.SampleMethods import MCS
import numpy as np
import matplotlib.pyplot as plt
from UQpy.Surrogates import *
from UQpy.SampleMethods import MCS
distribution1=Normal(1e5, 2*1e4)
distribution2=Uniform(0.01, 0.89)


training_sampling = MCS(dist_object=JointInd(marginals=[distribution1, distribution2]), nsamples=20)
training_samples=training_sampling.samples

model_serial_third_party=RunModel(samples=training_samples,  model_script='PythonAsThirdParty_model_2D.py',
    input_template='elastic_contact_sphere.py', var_names=['k', 'f0'],
    output_script='process_3rd_party_output.py', model_object_name='read_output')

maximum_displacement = model_serial_third_party.qoi_list[:20]

from UQpy.Surrogates import PCE, PolyChaosRidge, Polynomials

polys = Polynomials(dist_object=JointInd(marginals=[distribution1, distribution2]), degree=2)
lstsq = PolyChaosRidge(poly_object=polys)
pce = PCE(method=lstsq)

pce.fit(training_samples,np.array(maximum_displacement).reshape(20,1))

# Exercise 2 


### Kriging (Gaussian process)


Build a Kriging surrogate for 1-D contact sphere model. In the indentation test model randomness is assumed in the parameter $k$ which is assumed to follow a Lognormal distribution, with the parameters of the underlying Gaussian distribution being: $\mu$ = 1e5 and standard deviation $\sigma=2e4$. The parameters of the lognormal distribution in this case are $s=0.19804$ and $scale=98058.0675$ ($loc$=0.0). Build a Kriging surrogate to approximate the response function of the maximum displacement at the identation point. 

### Step 1

Create a distribution object for the random variable (see Day 1 activities).

In [1]:
# Solution
from UQpy.Distributions import Lognormal
dist1 = Lognormal(s=0.19804, scale=98058.0675, loc=0.0) 

### Step 2

Create a $\texttt{LHS}$ object (see Day 1 activities):
- generate 20 realizations of $r_0$ that will serve as our training input set.
- generate 30 realizations of $r_0$ that will serve as our validation input set.

In [2]:
# Solution
from UQpy.SampleMethods import LHS
training_sampling = LHS(dist_object=dist1, nsamples=20)
training_samples = training_sampling.samples.reshape(20,1)

validation_sampling = LHS(dist_object=dist1, nsamples=30)
validation_samples = validation_sampling.samples.reshape(30,1)

### Step 3

Run the 1-D contact sphere model using $\texttt{RunModel}$ module of $\texttt{UQpy}$ (see Day 1 activities):
- for the training input set.
- for the validation input set.

In [3]:
# Solution
from UQpy.RunModel import RunModel
model_serial_third_party=RunModel(model_script='PythonAsThirdParty_model.py', model_object_name='run_model',
    input_template='elastic_contact_sphere_1D.py', var_names=['k'],
    output_script='process_3rd_party_output.py')

model_serial_third_party.run(samples=training_samples)
qoi = model_serial_third_party.qoi_list

maximum_displacement=list(map(abs, qoi[:20]))

model_serial_third_party_validation=RunModel(model_script='PythonAsThirdParty_model.py', model_object_name='run_model',
    input_template='elastic_contact_sphere_1D.py', var_names=['k'],
    output_script='process_3rd_party_output.py')

model_serial_third_party_validation.run(samples=validation_samples)
maximum_displacement_validation=list(map(abs, qoi[:30]))

### Step 4

Create the Kriging surrogate to approximate the response function (i.e., maximum displacement at the identation point). 

### Step 4.1:

Import library  $\texttt{Kriging}$ from $\texttt{UQpy.Surrogates}$ module.

In [4]:
# Solution
from UQpy.Surrogates import Kriging

### Step 4.2:

Instantiate a $\texttt{Kriging}$ object. Select:

1. Linear regression model to evaluate the basis functions and their coefficients.
2. Gaussian correlation model to define similarity between samples.
3. 20 times the Maximum Likelihood optimization problem to be solved with a random starting point.

In [5]:
# Solution
K = Kriging(reg_model='Linear', corr_model='Gaussian', nopt=20, corr_model_params=[1])

### Step 4.3:

Calculate the hyperparameters using the $\texttt{fit}$ method of the $\texttt{Kriging}$ object. Provide the training data set (input-output pairs). Print the regression coefficients.

In [6]:
# Solution
K.fit(samples=training_samples, values=maximum_displacement)
print(K.beta)

[[0.05959993]
 [0.12781454]]


### Step 4.4:

Predic the response for the validation input set using the $\texttt{predict}$ method of the $\texttt{Kriging}$ object. Print the variance of the Kriging prediction.

In [7]:
# Solution
from UQpy.SampleMethods import MCS
prediction_sampling=MCS(dist_object=[dist1], nsamples=1000,  verbose=True)
prediction_results=K.predict(prediction_sampling.samples)

print(K.err_var)

UQpy: Running Monte Carlo Sampling.
UQpy: Monte Carlo Sampling Complete.
[1.00241701]


## Activities

1. Build the Kriging surrogate for the 1-D contact sphere model for:
    - For 100 training data.
    - Quadratic regression model
    - Exponential correlation function
    Compare the results.

2. Build a Kriging surrogate for 1-D Boucwen model (Exercise 1). In this case,  $r_{0}\sim \mathcal{U}(0.5, 3.5)$. Compare the results obtained with the PCE surrogate.

In [8]:
# Solution
training_sampling = LHS(dist_object=dist1, nsamples=100)
training_samples = training_sampling.samples

model_serial_third_party=RunModel(model_script='PythonAsThirdParty_model.py',
    input_template='elastic_contact_sphere_1D.py', var_names=['k'],
    output_script='process_3rd_party_output.py')

model_serial_third_party.run(samples=training_samples.reshape(-1,1))
qoi = model_serial_third_party.qoi_list

maximum_displacement=qoi[:100]

K1 = Kriging(reg_model='Quadratic', corr_model='Exponential', nopt=20, corr_model_params=[1])
K1.fit(samples=samples, values=maximum_displacement)

ValueError: 
UQpy: var_names must have the same length as the number of variables (i.e. len(var_names) = len(samples[0]).


In [ ]:
from UQpy.Distributions import Uniform
from UQpy.SampleMethods import MCS
from UQpy.RunModel import RunModel
from UQpy.Surrogates import Kriging

distribution=Uniform(0.5, 3.0)

training_sampling = MCS(dist_object=distribution, nsamples=30)
training_samples=training_sampling.samples

boucwen = RunModel(model_script='model_1D.py', model_object_name='boucwen_runmodel', var_names=['r0'])
boucwen.run(samples=training_samples)

maximum_displacement=boucwen.qoi_list[:30]

K2 = Kriging(reg_model='Linear', corr_model='Gaussian', nopt=20, corr_model_params=[1])
K2.fit(samples=training_samples, values=maximum_displacement)
